In [1]:
import torch as pt
import torch.nn as nn
from torch.utils.data import random_split
from src.dataset import MolDataset
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.metrics import f1_score, cohen_kappa_score
import sys

/home/dima/miniconda3/envs/ctpesto/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt 

%matplotlib inline
%config InlineBackend.figure_format =   'svg'

In [3]:
pt.manual_seed(1158)
device = "cuda" if pt.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Dataset and Net

In [4]:
N = 18000
dataset = MolDataset("../data/preprocessed/X.pk", 
                     y_datafile="../data/preprocessed/Y.pk", normal = True)
train_dataset, test_dataset = random_split(dataset, [len(dataset) - N, N])

In [5]:
len(train_dataset)

52711

In [6]:
dataset[33][0]

array([ 3.3105030e-03,  3.1218042e-03,  1.4059457e-01,  6.9875127e-01,
        0.0000000e+00,  6.6563778e-02,  3.1595469e-02,  3.5106074e-02,
        5.6169719e-02,  0.0000000e+00,  3.8616683e-02,  2.3026923e-02,
        2.9138044e-01,  1.4855877e-03,  3.5106074e-03,  1.0531822e-02,
        0.0000000e+00,  4.7062942e-01,  3.7212440e-01,  4.2127289e-02,
       -1.6148794e-04,  1.8255159e-01,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  3.5106074e-03,  0.0000000e+00,  3.5106074e-03,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
      

In [7]:
print(len(dataset)/np.sum(dataset[:][1], axis=0))

0.5257127987807144


In [8]:
np.sum(test_dataset[:][1] ==1)

740

In [26]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(66, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
                        nn.ReLU(),
            nn.Linear(128, 128),
                        nn.ReLU(),
            nn.Linear(128, 128),
                        nn.ReLU(),
            nn.Linear(128, 128),
                        nn.ReLU(),
            nn.Linear(128, 128),
                        nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    

    

In [27]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=66, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_features=128, bias=True)
    (9): ReLU()
    (10): Linear(in_features=128, out_features=128, bias=True)
    (11): ReLU()
    (12): Linear(in_features=128, out_features=128, bias=True)
    (13): ReLU()
    (14): Linear(in_features=128, out_features=64, bias=True)
    (15): ReLU()
    (16): Linear(in_features=64, out_features=3, bias=True)
  )
)


#  Train

## Example

In [ ]:
X.shape

In [28]:
ids = [8,2100,11111, 3674,9867]

X1 = train_dataset[ids][0]
X = pt.tensor(X1, device=device)
treu_y = dataset[ids][1]

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")
# print(f"True classes: {treu_y}")

In [29]:
pred_probab

tensor([[0.3035, 0.3596, 0.3370],
        [0.3035, 0.3596, 0.3370],
        [0.3035, 0.3596, 0.3370],
        [0.3035, 0.3596, 0.3370],
        [0.3035, 0.3596, 0.3370]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [19]:
pred_probab.argmax(1)

tensor([2, 2, 2, 1, 2], device='cuda:0')

In [20]:
treu_y

array([0, 1, 2, 2, 2])

## Parameters of training

In [31]:
# loss_fn = nn.BCEWithLogitsLoss(weight=class_weights)
class_weights = pt.Tensor([1, 1, 0.05])
class_weights.to(device)
# weight=class_weights,reduction='mean'
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)
optimizer = pt.optim.Adam(model.parameters())

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, ycls,  Y) in enumerate(dataloader):
        # Compute prediction and loss
        x = X.to(device)
        pred = model(x) 
        # p = pt.squeeze(pred, dim=1)
        yyy = pt.Tensor(ycls).type(pt.LongTensor).to(device)
        loss = loss_fn(pred, yyy)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 25 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            # print(f"score {score}")



def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    score = 0
    with pt.no_grad():
        for X, ycls, Y in dataloader:
            x = X.to(device)
            pred = model(x) 
            yyy = pt.Tensor(ycls).type(pt.LongTensor).to(device)
            # loss = loss_fn(p, yyy)
            test_loss += loss_fn(pred, yyy)
            
                        #convert to classes
            pred_probab = nn.Softmax(dim=1)(pred)
            pred_cats = np.array(pred_probab.argmax(1).cpu())  
            # # score = f1_score(pred_cats, ycls, average=None)
            # score = cohen_kappa_score(pred_cats, ycls)
            ts = f1_score(pred_cats, ycls, average=None)
            ts2 = cohen_kappa_score(pred_cats, ycls)
            # # print(ts.shape)
            score += ts2


    test_loss /= num_batches
    #     correct /= size
    #     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Avg loss: {test_loss:>8f} score: {score/52.}\n")
    return(score/52.)


In [ ]:
epochs = 45
# train model
train_dataloader = DataLoader(train_dataset, batch_size=1024,
                              shuffle=True, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=1024,
                             shuffle=True, pin_memory=True)
scores = []
for t in range(epochs):
    sys.stdout.write(f"{t} ") 
    print(f"Epoch {t + 1}\n-------------------------------")
    # logger.store_progress(0, is_train=True, epoch=t+1)
    train_loop(train_dataloader, model, loss_fn, optimizer)
    s = test_loop(test_dataloader, model, loss_fn)
    scores.append(s)
print("Done!")

0 Epoch 1
-------------------------------
loss: 1.088062  [    0/52711]
loss: 0.367582  [25600/52711]
loss: 0.293686  [51200/52711]
Avg loss: 0.318139 score: 0.0

1 Epoch 2
-------------------------------
loss: 0.338364  [    0/52711]
loss: 0.268851  [25600/52711]
loss: 0.269277  [51200/52711]
Avg loss: 0.304904 score: 0.0

2 Epoch 3
-------------------------------
loss: 0.326655  [    0/52711]
loss: 0.272389  [25600/52711]
loss: 0.285632  [51200/52711]
Avg loss: 0.302840 score: 0.0

3 Epoch 4
-------------------------------
loss: 0.288717  [    0/52711]
loss: 0.284730  [25600/52711]
loss: 0.300795  [51200/52711]
Avg loss: 0.302762 score: 0.0

4 Epoch 5
-------------------------------
loss: 0.284776  [    0/52711]
loss: 0.319952  [25600/52711]
loss: 0.295664  [51200/52711]
Avg loss: 0.303807 score: 0.0

5 Epoch 6
-------------------------------
loss: 0.308664  [    0/52711]
loss: 0.267728  [25600/52711]
loss: 0.277807  [51200/52711]
Avg loss: 0.302496 score: 0.0

6 Epoch 7
------------

In [ ]:
plt.plot(scores)
plt.ylim([-0.00001,0.1])

In [ ]:
pt.save(model.state_dict(), "model.pt")

In [ ]:
with pt.no_grad():
    for X, Y in train_dataloader:
        x = X.to(device)
        y = Y[:,None,:].to(device)

In [ ]:
logits = model(x)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)

In [ ]:
x.shape

In [ ]:
logits[0:10]

In [ ]:
y[0:8,:,:]